Following notebook demonstrates a quick way to predict sentiment of a writtent text with no prior label data training.

Hope you like it !

In [13]:
import os
import pandas as pd
import math as m
from time import gmtime, strftime
import numpy as np
import pyodbc
import sqlalchemy as sa 
import psycopg2
import re

In [4]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/home/rahul/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Initialize Sentiment Analyzer : An out of the box functaionality from NLTK library

In [5]:
sid = SentimentIntensityAnalyzer()

Let's do a quick check with analysis of one tweet and see how good this out of the box functionality works

In [6]:
AK_Tweet="Hon'ble LG agreed DISCOMS shud pay compensation 4 unscheduled power cuts 2 make them accountable. Fairplay n equity 2 consumers 2 be ensured"
print AK_Tweet

Hon'ble LG agreed DISCOMS shud pay compensation 4 unscheduled power cuts 2 make them accountable. Fairplay n equity 2 consumers 2 be ensured


Let's check polarity score for the above tweet

In [7]:
print sid.polarity_scores(AK_Tweet)

{'neg': 0.174, 'neu': 0.725, 'pos': 0.101, 'compound': -0.128}


The above polaity score suggest most of the words used are neutral in nature however overall compound score is "negative"

Also, notice that negative score is higher than positive score suggeting me that its a negative tweet.

Which is true. Isn't it !

Let's try removing stopwords & numbers as neither conveys sentiments and see if it improves the scores.

In [18]:
stop_words = set(stopwords.words('english'))
words = AK_Tweet.split()
AK_Tweet_WithoutStopWordsnNumbers = ''
for r in words:
    if not r in stop_words:
        AK_Tweet_WithoutStopWordsnNumbers = AK_Tweet_WithoutStopWordsnNumbers + ' ' +  r
AK_Tweet_WithoutStopWordsnNumbers = re.sub(r'\d+', '', AK_Tweet_WithoutStopWordsnNumbers)
print AK_Tweet_WithoutStopWordsnNumbers

 Hon'ble LG agreed DISCOMS shud pay compensation  unscheduled power cuts  make accountable. Fairplay n equity  consumers  ensured


In [19]:
print sid.polarity_scores(AK_Tweet_WithoutStopWordsnNumbers)

{'neg': 0.193, 'neu': 0.695, 'pos': 0.112, 'compound': -0.128}


Let's say you have a dataframe with text to be analysed likw :

* movie reviews
* call notes
* tweets 
* facebook posts etc

and all the posts are unlabeled i.e no prior knowledge of sentiment available. 

Following code can do the trick and give a sentiment inidicator or classify text as positive of negative !!! 


In [9]:
from urllib2 import Request, urlopen
import json
data = pd.read_json("//home//rahul//Downloads//Amazon_Instant_Video_5.json", lines=True)
data.head(2)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000H00VBQ,"[0, 0]",2,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200
1,B000H00VBQ,"[0, 0]",5,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400


Classify product reviews & Get Positive reviews only

In [12]:
for index, row in data.head(20).iterrows():
    ss = sid.polarity_scores(row['reviewText'])
    if((ss['pos']>0)&(ss['pos']>ss['neg'])):
        print '\nPositive Reviews'
        print row['reviewText']


Positive Reviews
I highly recommend this series. It is a must for anyone who is yearning to watch "grown up" television. Complex characters and plots to keep one totally involved. Thank you Amazin Prime.

Positive Reviews
This show always is excellent, as far as british crime or mystery showsgoes this is one of the best ever made.  The stories are well done and the acting is top notch with interesting twists in the realistic and brutal storylines.  This show pulls no punches as it enters into the twisted minds of criminals and the profiler psychiatrist who helps out in a northern english city police force. The show looks like it is shot in Manchester but it is called by another name in the show.  One episode is not on this disc the excellent 'prayer of the bone" which is on a seperate disc.  Still crime shows don't get much better than this one on either side of the ocean.  It's just a great show that never has had a less than well made episode. Unfortunately like all British shows yo

Fetch Negative Reviews

In [13]:
for index, row in data.head(40).iterrows():
    ss = sid.polarity_scores(row['reviewText'])
    if((ss['neg']>0)&(ss['pos']<ss['neg'])):
        print '\nNegative Reviews'
        print row['reviewText']


Negative Reviews
I had big expectations because I love English TV, in particular Investigative and detective stuff but this guy is really boring. It didn't appeal to me at all.

Negative Reviews
This one is a real snoozer. Don't believe anything you read or hear, it's awful. I had no idea what the title means. Neither will you.

Negative Reviews
Mysteries are interesting.  The tension between Robson and the tall blond is good but not always believable.  She often seemed uncomfortable.

Negative Reviews
I discovered this series quite by accident. Having watched and appreciated Masterpiece Contemporary: Place of Execution, I was keen to read the novel (which inspired the TV adaptation) by Val McDermid. The novel was very well-written, and a nail-biting suspense thriller. Then I discovered that Val McDermid wrote other novels as well, and a couple of them inspired the TV crime drama Wire in the Blood.I finished watching all of Season 1 and have become a fan of this gritty crime drama tha